In [6]:
# 이미지 파일명 덮어쓰기 방지
import os
import csv
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 이미지 저장 폴더 설정
thumbnail_folder = "product_images"
detail_folder = "product_detail_images"
os.makedirs(thumbnail_folder, exist_ok=True)
os.makedirs(detail_folder, exist_ok=True)

# CSV 설정
csv_filename = "reviews_including_empty.csv"
csv_columns = ["상품 이름", "이미지 파일명", "ID", "별점", "리뷰 내용"]
if not os.path.exists(csv_filename):
    with open(csv_filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(csv_columns)

# Selenium 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://www.mychef.kr/product/list?depth1=All")
time.sleep(3)

page = 1
clicked_urls = set()
global_index = 1  # 이미지 파일명 중복 방지를 위한 전역 인덱스

try:
    while True:
        print(f"\n {page} 페이지 크롤링 시작")

        product_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href^='/product/view?productCd=']"))
        )
        product_urls = list(set([p.get_attribute("href") for p in product_elements]))

        for product_url in product_urls:
            if product_url in clicked_urls:
                continue

            try:
                driver.get(product_url)
                time.sleep(2)

                # 상품 이름
                try:
                    product_name = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".goods-tit"))
                    ).text.strip()
                except:
                    product_name = "상품 이름 없음"

                # # 썸네일 이미지 저장
                # try:
                #     img = driver.find_element(By.CSS_SELECTOR, "img.lozad")
                #     thumb_url = img.get_attribute("src")
                #     thumb_name = os.path.join(thumbnail_folder, f"{global_index}_image.jpg")
                #     if thumb_url.startswith("http"):
                #         with open(thumb_name, "wb") as f:
                #             f.write(requests.get(thumb_url).content)
                #         print(f"썸네일 저장 완료: {thumb_name}")
                #     else:
                #         thumb_name = "이미지없음"
                # except:
                #     print("썸네일 저장 실패")
                #     thumb_name = "이미지없음"
                try:
                    img = driver.find_element(By.CSS_SELECTOR, "img.lozad")
                    thumb_url = img.get_attribute("src") or img.get_attribute("data-src") or img.get_attribute("data-original")
                    if thumb_url and thumb_url.startswith("http"):
                        response = requests.get(thumb_url, timeout=5)
                        if response.status_code == 200 and 'image' in response.headers.get('Content-Type', ''):
                            thumb_name = os.path.join(thumbnail_folder, f"{global_index}_image.jpg")
                            with open(thumb_name, "wb") as f:
                                f.write(response.content)
                            print(f"썸네일 저장 완료: {thumb_name}")
                        else:
                            print("이미지 응답 실패 또는 이미지 아님")
                            thumb_name = "이미지없음"
                    else:
                        print("유효한 이미지 URL 없음")
                        thumb_name = "이미지없음"
                except Exception as e:
                    print("썸네일 저장 실패:", e)
                    thumb_name = "이미지없음"

                # 상세 이미지 (첫 번째) 저장
                try:
                    detail_img = driver.find_element(By.CSS_SELECTOR, "div.inner-content.productCont img")
                    detail_url = detail_img.get_attribute("src")
                    detail_name = os.path.join(detail_folder, f"{global_index}_detail.jpg")
                    if detail_url.startswith("http"):
                        with open(detail_name, "wb") as f:
                            f.write(requests.get(detail_url).content)
                        print(f"상세 이미지 저장 완료: {detail_name}")
                except:
                    print("상세 이미지 없음")

                # 리뷰 수집
                review_page = 1
                seen_ids = set()
                all_reviews = []

                while True:
                    time.sleep(2)
                    reviews = driver.find_elements(By.CSS_SELECTOR, "li.review-item")
                    new = []

                    for r in reviews:
                        rid = r.get_attribute("data-id")
                        if rid in seen_ids:
                            continue
                        seen_ids.add(rid)

                        try:
                            user = r.find_element(By.CSS_SELECTOR, ".name.user-name").text.strip()
                            uid = user[:5]
                        except:
                            uid = ""

                        try:
                            star = r.find_element(By.CSS_SELECTOR, ".pointReviewContent").get_attribute("style")
                            match = re.search(r'width:\s*(\d+(?:\.\d+)?)px', star)
                            rating = round(float(match.group(1)) / 19, 1) if match else "-"
                        except:
                            rating = ""

                        try:
                            text = r.find_element(By.CSS_SELECTOR, "p.review.text span").text.strip()
                        except:
                            text = ""

                        new.append([product_name, f"{global_index}_image.jpg", uid, rating, text])

                    if new:
                        all_reviews.extend(new)
                        print(f"리뷰 {len(new)}개 수집됨")

                    # 다음 리뷰 페이지
                    try:
                        review_page += 1
                        driver.execute_script(f"refreshReviewPaging('{review_page}')")
                        WebDriverWait(driver, 5).until(
                            lambda d: any(
                                r.get_attribute("data-id") not in seen_ids
                                for r in d.find_elements(By.CSS_SELECTOR, "li.review-item")
                            )
                        )
                    except:
                        print("리뷰 끝")
                        break

                # 리뷰 없을 경우 기본 저장
                if not all_reviews:
                    all_reviews.append([product_name, f"{global_index}_image.jpg", "", "", ""])

                # CSV 저장
                with open(csv_filename, "a", newline="", encoding="utf-8-sig") as f:
                    writer = csv.writer(f)
                    writer.writerows(all_reviews)

                print(f"저장 완료: {len(all_reviews)}개")
                clicked_urls.add(product_url)
                global_index += 1  # 다음 상품 인덱스 증가
                driver.back()
                time.sleep(2)

            except Exception as e:
                print(f"❌ 상품 오류: {e}")
                continue

        # 다음 상품 페이지로 이동
        try:
            page += 1
            driver.execute_script(f"cmPageMove('{page}')")
            time.sleep(3)
        except:
            print("모든 상품 페이지 완료")
            break

except Exception as e:
    print(f"❌ 전체 오류 발생: {e}")

driver.quit()



 1 페이지 크롤링 시작
썸네일 저장 완료: product_images\1_image.jpg
상세 이미지 저장 완료: product_detail_images\1_detail.jpg
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 2개 수집됨
리뷰 끝
저장 완료: 17개
썸네일 저장 완료: product_images\2_image.jpg
상세 이미지 저장 완료: product_detail_images\2_detail.jpg
리뷰 2개 수집됨
리뷰 끝
저장 완료: 2개
썸네일 저장 완료: product_images\3_image.jpg
상세 이미지 저장 완료: product_detail_images\3_detail.jpg
리뷰 5개 수집됨
리뷰 끝
저장 완료: 5개
썸네일 저장 완료: product_images\4_image.jpg
상세 이미지 저장 완료: product_detail_images\4_detail.jpg
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 5개 수집됨
리뷰 3개 수집됨
리뷰 끝
저장 완료: 68개


KeyboardInterrupt: 

In [22]:
!pip uninstall -y moviepy
!pip install moviepy==1.0.3


Found existing installation: moviepy 2.1.2
Uninstalling moviepy-2.1.2:
  Successfully uninstalled moviepy-2.1.2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110755 sha256=f14aeed68f7faa0d0bf9e76a57e7088fa227a2826714e823c6bd6c1fe570b0d6
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\df\ba\4b\0917fc0c8833c8ba7016565fc975b74c67bc8610806e930272
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [23]:
import sys
print(sys.executable)


C:\anaconda\python.exe


In [24]:
import moviepy
print(moviepy.__file__)


C:\anaconda\Lib\site-packages\moviepy\__init__.py


In [28]:
!pip install decorator==4.4.2


In [1]:
from moviepy.editor import VideoFileClip, clips_array

print("moviepy.editor 정상 작동!")


moviepy.editor 정상 작동!


In [2]:
from moviepy.editor import VideoFileClip, clips_array

clip1 = VideoFileClip("C:/Users/admin/Videos/Captures/video1.mp4")
clip2 = VideoFileClip("C:/Users/admin/Videos/Captures/video2.mp4")

min_dur = min(clip1.duration, clip2.duration)
start = max(min_dur - 30, 0)

clip1 = clip1.subclip(start, min_dur).resize(height=360)
clip2 = clip2.subclip(start, min_dur).resize(height=360)

final = clips_array([[clip1, clip2]])
final.write_videofile("output_combined.mp4", fps=15, codec="libx264", preset="ultrafast", audio_codec="aac")


Moviepy - Building video output_combined.mp4.
MoviePy - Writing audio in output_combinedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_combined.mp4



Moviepy - Done !
Moviepy - video ready output_combined.mp4
